In [12]:
import os
from datetime import datetime
from zoneinfo import ZoneInfo  # Python 3.9+
from pathlib import Path

import earthaccess
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
from shapely.ops import transform as shp_transform
import pyproj
import numpy as np

## 0.1 Download Data (Summer & Nighttime) from NASA Earthdata

In [17]:
"""
Download ECOSTRESS ECO_L2T_LSTE nighttime summer LST for Philadelphia

Requirements:
pip install earthaccess geopandas shapely

"""

import os
import re
from datetime import datetime
from zoneinfo import ZoneInfo

import geopandas as gpd
import earthaccess


# ---------------- User-configurable parameters ----------------

CITY_BOUNDARY_FILE = "data/City_Limits.shp"   # City boundary file (Philly)

OUTPUT_DIR = "data/ecostress_philly_summer_night"  # Folder to save downloaded files

# Nighttime window in local (Philadelphia) time, e.g. 18:00–02:00 next day
NIGHT_START_HOUR = 19
NIGHT_END_HOUR = 4

# Summer months
SUMMER_MONTHS = [7, 8, 9]   

# Search date range (UTC dates, inclusive)
START_DATE = "2018-07-01"
END_DATE = "2025-12-31"

# ECOSTRESS product
ECOSTRESS_PRODUCT = "ECO_L2T_LSTE"

# Philadelphia local timezone
PHILLY_TZ = ZoneInfo("America/New_York")


# ---------------- Helper functions ----------------

def get_city_bbox(city_file):
    """
    Bounding Box for filtering
    """
    gdf = gpd.read_file(city_file)
   
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    minx, miny, maxx, maxy = gdf.total_bounds
    # earthaccess needs [west, south, east, north]
    return (minx, miny, maxx, maxy)


def is_night_local(local_dt):
    """
    Check if a local datetime in Philadelphia is within the nighttime window.

    Args:
        local_dt (datetime): timezone-aware datetime in PHILLY_TZ

    Returns:
        bool
    """
    month = local_dt.month
    hour = local_dt.hour

    if month not in SUMMER_MONTHS:
        return False

    # Handle windows that may cross midnight (e.g., 18:00–02:00)
    if NIGHT_START_HOUR < NIGHT_END_HOUR:

        return NIGHT_START_HOUR <= hour < NIGHT_END_HOUR
    else:

        return (hour >= NIGHT_START_HOUR) or (hour < NIGHT_END_HOUR)


def get_utc_time_from_filename(granule):
    """
    Parse UTC datetime from ECOSTRESS file name, e.g. 20190708T094058.

    File name like:
        ECOv002_L2T_LSTE_41552_00718TVK_20251103T095416_0713_02

    Args:
        granule: granule from earthaccess search_data  

    Returns:
        timezone-aware datetime in UTC, or None if not found.
    """
    try:
        
        links = granule.data_links()
        if not links:
            return None
        fname = os.path.basename(links[0])
    except Exception:
        return None

    
    m = re.search(r"(\d{8}T\d{6})", fname)
    if not m:
        return None

    timestr = m.group(1)  # e.g. "20190708T094058"
    try:
        dt_naive = datetime.strptime(timestr, "%Y%m%dT%H%M%S")
    except ValueError:
        return None

    # 设为 UTC 时间 / make it UTC
    return dt_naive.replace(tzinfo=ZoneInfo("UTC"))


def filter_granules_by_filename_time(granules, debug_print=False, debug_n=5):
    """
    Use timestamp in file name to filter granules by summer & nighttime.

    Returns:
        list of granules that match the criteria
    """
    selected = []
    sample_count = 0

    for g in granules:
        utc_dt = get_utc_time_from_filename(g)
        if utc_dt is None:
            continue

        local_dt = utc_dt.astimezone(PHILLY_TZ)

        # Print for chekcing case
        if debug_print and sample_count < debug_n:
            try:
                fname = os.path.basename(g.data_links()[0])
            except Exception:
                fname = "<unknown>"
            print(f"Sample granule: {fname}")
            print(f"    UTC time   : {utc_dt}")
            print(f"    Philly time: {local_dt}")
            sample_count += 1

        if is_night_local(local_dt):
            selected.append(g)

    return selected


def ensure_output_dir(path):
    """Create output directory if it does not exist."""
    os.makedirs(path, exist_ok=True)


# ---------------- Main workflow ----------------

def main():
    # 1. Read city boundary and compute bounding box
    print("Reading city boundary and computing bounding box ...")
    bbox = get_city_bbox(CITY_BOUNDARY_FILE)
    print("Bounding box (west, south, east, north):", bbox)

    # 2. Login to Earthdata
    print("Logging in to Earthdata ...")
    earthaccess.login(persist=True)

    # 3. Search ECOSTRESS ECO_L2T_LSTE granules over the city bbox and date range
    print(f"Searching {ECOSTRESS_PRODUCT} from {START_DATE} to {END_DATE} ...")
    results = earthaccess.search_data(
        short_name=ECOSTRESS_PRODUCT,
        bounding_box=bbox,
        temporal=(START_DATE, END_DATE),
    )

    print("Total granules over city (all months & times):", len(results))

    if not results:
        print("No granules found for this bbox and date range. "
              "Check your city boundary and temporal range.")
        return

    # 4. Filter by summer months and nighttime overpass time
    print("Filtering granules to summer nighttime overpasses "
          "(local time in Philadelphia, using filename timestamp) ...")
    night_summer_granules = filter_granules_by_filename_time(
        results, debug_print=True, debug_n=5
    )
    print("Granules matching summer nighttime criteria:", len(night_summer_granules))

    if not night_summer_granules:
        print("No granules matched the summer nighttime criteria.")
        print("Check the sample local times above and adjust:")
        print("  - SUMMER_MONTHS")
        print("  - NIGHT_START_HOUR / NIGHT_END_HOUR")
        return

    # 5. Download selected granules into OUTPUT_DIR
    ensure_output_dir(OUTPUT_DIR)

    print(f"Downloading {len(night_summer_granules)} granules to folder: {OUTPUT_DIR}")
    downloaded_files = earthaccess.download(
        night_summer_granules,
        local_path=OUTPUT_DIR,
    )

    print("Download finished.")
    print("Downloaded files:")
    for f in downloaded_files:
        print("  ", f)

    print("\nNote: ECO_L2T_LSTE granules are usually HDF5 (.h5).")
    print("If you need GeoTIFF (.tif), you can later convert them using GDAL/rasterio.")


if __name__ == "__main__":
    main()


Reading city boundary and computing bounding box ...
Bounding box (west, south, east, north): (-75.2803067548922, 39.8674718597081, -74.9557485615157, 40.1379348411103)
Logging in to Earthdata ...
Searching ECO_L2T_LSTE from 2018-07-01 to 2025-12-31 ...
Total granules over city (all months & times): 3134
Filtering granules to summer nighttime overpasses (local time in Philadelphia, using filename timestamp) ...
Sample granule: ECOv002_L2T_LSTE_00344_009_18TVK_20180728T224024_0712_01_water.tif
    UTC time   : 2018-07-28 22:40:24+00:00
    Philly time: 2018-07-28 18:40:24-04:00
Sample granule: ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_water.tif
    UTC time   : 2018-07-29 05:09:04+00:00
    Philly time: 2018-07-29 01:09:04-04:00
Sample granule: ECOv002_L2T_LSTE_00363_005_18TWK_20180730T041714_0712_01_water.tif
    UTC time   : 2018-07-30 04:17:14+00:00
    Philly time: 2018-07-30 00:17:14-04:00
Sample granule: ECOv002_L2T_LSTE_00405_007_18TWK_20180801T204933_0712_01_water

QUEUEING TASKS | :   0%|          | 0/2696 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2696 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2696 [00:00<?, ?it/s]

Download finished.
Downloaded files:
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_water.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_cloud.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_view_zenith.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_height.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_QC.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_LST.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_LST_err.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_EmisWB.tif
   data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00363_005_18TWK_20180730T041714_0712_01_water.tif
   data\ecost

## 0.2 Filter the LST Data and Change the File Name to Local Time

In [35]:
import os
import shutil
import re
from datetime import datetime
from zoneinfo import ZoneInfo

INPUT_DIR = "data\ecostress_philly_summer_night"

OUTPUT_DIR = "data\ecostress_philly_LST_localtime"

# Philadelphia timezone
PHILLY_TZ = ZoneInfo("America/New_York")

# ========================================================

def extract_utc_timestr(fname):
    """
    Extract UTC timestamp (YYYYMMDDTHHMMSS) from filename.
    """
    m = re.search(r"(\d{8}T\d{6})", fname)
    if not m:
        return None
    return m.group(1)


def copy_and_rename_lst(src_dir, dst_dir):
    """
    New file name：
        YYYY-MM-DD_HH-MM-SS_LST.tif
        e.g. 2018-07-29_01-09-04_LST.tif
    """
    os.makedirs(dst_dir, exist_ok=True)

    count = 0
    skipped = 0

    for root, dirs, files in os.walk(src_dir):
        for fname in files:
            low = fname.lower()

            # Slect "_lst.tif" files
            if not low.endswith("_lst.tif"):
                continue
            if "lst_err" in low:
                continue

            utc_str = extract_utc_timestr(fname)
            if utc_str is None:
                print(f"[WARN] Cannot find UTC timestamp in filename, skip: {fname}")
                skipped += 1
                continue

            # parse as UTC datetime
            dt_utc = datetime.strptime(utc_str, "%Y%m%dT%H%M%S").replace(tzinfo=ZoneInfo("UTC"))
            # convert to Philadelphia local time
            dt_local = dt_utc.astimezone(PHILLY_TZ)

            # Change to 2018-07-29_01-09-04
            local_str_pretty = dt_local.strftime("%Y-%m-%d_%H-%M-%S")

            # new name：YYYY-MM-DD_HH-MM-SS_LST.tif
            new_name = f"{local_str_pretty}_LST.tif"
            dst_path = os.path.join(dst_dir, new_name)

            if os.path.exists(dst_path):
                i = 1
                while True:
                    alt_name = f"{local_str_pretty}_LST_{i:02d}.tif"
                    alt_path = os.path.join(dst_dir, alt_name)
                    if not os.path.exists(alt_path):
                        dst_path = alt_path
                        new_name = alt_name
                        break
                    i += 1

            src_path = os.path.join(root, fname)
            shutil.copy2(src_path, dst_path)

            count += 1
            print(f"Copied: {src_path}")
            print(f"   -->  {dst_path}")

    print(f"\nDone. Total LST files copied & renamed: {count}")
    if skipped:
        print(f"Files skipped due to missing timestamp: {skipped}")


if __name__ == "__main__":
    copy_and_rename_lst(INPUT_DIR, OUTPUT_DIR)


Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00348_001_18TVK_20180729T050904_0712_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2018-07-29_01-09-04_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00363_005_18TWK_20180730T041714_0712_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2018-07-30_00-17-14_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00455_017_18TVK_20180805T021832_0712_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2018-08-04_22-18-32_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00455_017_18TWK_20180805T021832_0712_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2018-08-04_22-18-32_LST_01.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00455_018_18TWK_20180805T021924_0712_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2018-08-04_22-19-24_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_00501_012_18TVK_20180808T011902_0712_

Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28480_008_18TVK_20230715T073108_0710_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2023-07-15_03-31-08_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28480_008_18TVK_20230715T073108_0712_02_LST.tif
   -->  data\ecostress_philly_LST_localtime\2023-07-15_03-31-08_LST_01.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28480_008_18TWK_20230715T073108_0710_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2023-07-15_03-31-08_LST_02.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28480_008_18TWK_20230715T073108_0712_02_LST.tif
   -->  data\ecostress_philly_LST_localtime\2023-07-15_03-31-08_LST_03.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28491_005_18TWK_20230716T001251_0710_01_LST.tif
   -->  data\ecostress_philly_LST_localtime\2023-07-15_20-12-51_LST.tif
Copied: data\ecostress_philly_summer_night\ECOv002_L2T_LSTE_28491_005_18TWK_20230716T001251

## 0.3 Use Philly Boundary to Mask LST data

In [36]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

LST_DIR = "data\ecostress_philly_LST_localtime"
CITY_BOUNDARY_FILE = "data\City_Limits.geojson"
OUTPUT_DIR = "data\ecostress_philly_LST_clip"


def load_city_boundary(boundary_path):
    """
    Load city boundary and return:
        - GeoDataFrame with all features
        - A merged geometry (union of all polygons, if multiple exist)
    """
    gdf = gpd.read_file(boundary_path)
    boundary_geom = gdf.unary_union
    return gdf, boundary_geom   


def clip_raster_with_boundary(in_raster, out_raster, boundary_gdf):
    """
    Clip a single raster by city boundary.
    """
    with rasterio.open(in_raster) as src:
        if boundary_gdf.crs != src.crs:
            boundary_proj = boundary_gdf.to_crs(src.crs)
        else:
            boundary_proj = boundary_gdf

        geoms = [geom.__geo_interface__ for geom in boundary_proj.geometry]

        out_image, out_transform = mask(src, geoms, crop=True)

        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width":  out_image.shape[2],
            "transform": out_transform
        })

    with rasterio.open(out_raster, "w", **out_meta) as dest:
        dest.write(out_image)


def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Load city boundary
    boundary_gdf, _ = load_city_boundary(CITY_BOUNDARY_FILE)
    print("City boundary CRS:", boundary_gdf.crs)

    # Loop LST tif / List all LST tif files
    lst_files = [
        f for f in os.listdir(LST_DIR)
        if f.lower().endswith(".tif")
    ]
    lst_files.sort()
    print(f"Found {len(lst_files)} LST tif files.")

    for i, fname in enumerate(lst_files, start=1):
        in_path = os.path.join(LST_DIR, fname)
        name_no_ext, ext = os.path.splitext(fname)
        out_name = f"{name_no_ext}_clip{ext}"
        out_path = os.path.join(OUTPUT_DIR, out_name)

        print(f"[{i}/{len(lst_files)}] Clipping {fname} -> {out_name}")

        try:
            clip_raster_with_boundary(in_path, out_path, boundary_gdf)
        except Exception as e:
            print(f"  !! Failed to clip {fname}: {e}")

    print("\nAll done. Clipped rasters are saved in:")
    print(f"  {OUTPUT_DIR}")


if __name__ == "__main__":
    main()


City boundary CRS: EPSG:4326
Found 337 LST tif files.
[1/337] Clipping 2018-07-29_01-09-04_LST.tif -> 2018-07-29_01-09-04_LST_clip.tif
[2/337] Clipping 2018-07-30_00-17-14_LST.tif -> 2018-07-30_00-17-14_LST_clip.tif
[3/337] Clipping 2018-08-04_22-18-32_LST.tif -> 2018-08-04_22-18-32_LST_clip.tif
[4/337] Clipping 2018-08-04_22-18-32_LST_01.tif -> 2018-08-04_22-18-32_LST_01_clip.tif
[5/337] Clipping 2018-08-04_22-19-24_LST.tif -> 2018-08-04_22-19-24_LST_clip.tif
[6/337] Clipping 2018-08-07_21-19-02_LST.tif -> 2018-08-07_21-19-02_LST_clip.tif
[7/337] Clipping 2018-08-07_21-19-02_LST_01.tif -> 2018-08-07_21-19-02_LST_01_clip.tif
[8/337] Clipping 2018-08-08_20-27-26_LST.tif -> 2018-08-08_20-27-26_LST_clip.tif
[9/337] Clipping 2018-08-08_20-27-26_LST_01.tif -> 2018-08-08_20-27-26_LST_01_clip.tif
[10/337] Clipping 2018-09-06_03-19-54_LST.tif -> 2018-09-06_03-19-54_LST_clip.tif
[11/337] Clipping 2018-09-06_03-19-54_LST_01.tif -> 2018-09-06_03-19-54_LST_01_clip.tif
[12/337] Clipping 2018-09-07_

[102/337] Clipping 2020-09-20_01-59-26_LST.tif -> 2020-09-20_01-59-26_LST_clip.tif
[103/337] Clipping 2020-09-20_02-00-18_LST.tif -> 2020-09-20_02-00-18_LST_clip.tif
[104/337] Clipping 2020-09-20_02-00-18_LST_01.tif -> 2020-09-20_02-00-18_LST_01_clip.tif
[105/337] Clipping 2020-09-23_01-14-37_LST.tif -> 2020-09-23_01-14-37_LST_clip.tif
[106/337] Clipping 2020-09-23_01-14-37_LST_01.tif -> 2020-09-23_01-14-37_LST_01_clip.tif
[107/337] Clipping 2020-09-24_00-27-31_LST.tif -> 2020-09-24_00-27-31_LST_clip.tif
[108/337] Clipping 2020-09-24_00-27-31_LST_01.tif -> 2020-09-24_00-27-31_LST_01_clip.tif
[109/337] Clipping 2021-07-12_22-19-36_LST.tif -> 2021-07-12_22-19-36_LST_clip.tif
[110/337] Clipping 2021-07-16_20-47-29_LST.tif -> 2021-07-16_20-47-29_LST_clip.tif
[111/337] Clipping 2021-07-16_20-47-29_LST_01.tif -> 2021-07-16_20-47-29_LST_01_clip.tif
[112/337] Clipping 2021-07-17_03-17-48_LST.tif -> 2021-07-17_03-17-48_LST_clip.tif
[113/337] Clipping 2021-07-17_03-17-48_LST_01.tif -> 2021-07-17

[206/337] Clipping 2023-07-15_03-31-08_LST_01.tif -> 2023-07-15_03-31-08_LST_01_clip.tif
[207/337] Clipping 2023-07-15_03-31-08_LST_02.tif -> 2023-07-15_03-31-08_LST_02_clip.tif
[208/337] Clipping 2023-07-15_03-31-08_LST_03.tif -> 2023-07-15_03-31-08_LST_03_clip.tif
[209/337] Clipping 2023-07-15_20-12-51_LST.tif -> 2023-07-15_20-12-51_LST_clip.tif
[210/337] Clipping 2023-07-15_20-12-51_LST_01.tif -> 2023-07-15_20-12-51_LST_01_clip.tif
[211/337] Clipping 2023-07-15_20-13-43_LST.tif -> 2023-07-15_20-13-43_LST_clip.tif
[212/337] Clipping 2023-07-15_20-13-43_LST_01.tif -> 2023-07-15_20-13-43_LST_01_clip.tif
[213/337] Clipping 2023-07-16_02-42-58_LST.tif -> 2023-07-16_02-42-58_LST_clip.tif
[214/337] Clipping 2023-07-16_02-42-58_LST_01.tif -> 2023-07-16_02-42-58_LST_01_clip.tif
[215/337] Clipping 2023-07-19_01-55-05_LST.tif -> 2023-07-19_01-55-05_LST_clip.tif
[216/337] Clipping 2023-07-19_01-55-05_LST_01.tif -> 2023-07-19_01-55-05_LST_01_clip.tif
[217/337] Clipping 2023-07-19_01-55-05_LST_02

[306/337] Clipping 2025-07-22_23-02-25_LST.tif -> 2025-07-22_23-02-25_LST_clip.tif
[307/337] Clipping 2025-07-22_23-02-25_LST_01.tif -> 2025-07-22_23-02-25_LST_01_clip.tif
[308/337] Clipping 2025-07-23_22-14-00_LST.tif -> 2025-07-23_22-14-00_LST_clip.tif
[309/337] Clipping 2025-07-23_22-14-00_LST_01.tif -> 2025-07-23_22-14-00_LST_01_clip.tif
[310/337] Clipping 2025-07-26_21-25-24_LST.tif -> 2025-07-26_21-25-24_LST_clip.tif
[311/337] Clipping 2025-07-26_21-25-24_LST_01.tif -> 2025-07-26_21-25-24_LST_01_clip.tif
[312/337] Clipping 2025-07-30_19-48-04_LST.tif -> 2025-07-30_19-48-04_LST_clip.tif
[313/337] Clipping 2025-07-30_19-48-04_LST_01.tif -> 2025-07-30_19-48-04_LST_01_clip.tif
[314/337] Clipping 2025-08-26_02-43-17_LST.tif -> 2025-08-26_02-43-17_LST_clip.tif
[315/337] Clipping 2025-08-26_02-43-17_LST_01.tif -> 2025-08-26_02-43-17_LST_01_clip.tif
[316/337] Clipping 2025-08-29_01-54-30_LST.tif -> 2025-08-29_01-54-30_LST_clip.tif
[317/337] Clipping 2025-08-29_01-54-30_LST_01.tif -> 2025

## 0.4 Remove Problematic Data 

In [38]:
"""
Clean clipped ECOSTRESS LST rasters.

Steps:
1. Remove "long strip" rasters (very skinny width/height).

2. Remove rasters that are all NoData or constant value.

3. Copy remaining rasters into a new clean folder.

"""

import os
import shutil

import numpy as np
import rasterio


# ========== Path settings  ==========

# Input folder: clipped LST rasters (may contain long strips & empty rasters)

INPUT_DIR = "data\ecostress_philly_LST_clip"

# Output folder: cleaned LST rasters

OUTPUT_DIR = "data\ecostress_philly_LST_clean"

# Threshold for "long strip":
# if max(width/height, height/width) > ASPECT_RATIO_THRESHOLD -> treat as strip

ASPECT_RATIO_THRESHOLD = 3

# =================================================


def is_long_strip(width, height, ratio_thresh=ASPECT_RATIO_THRESHOLD):
    """
    Determine if a raster is a "long strip" based on width and height.

    Rule: max(width/height, height/width) > ratio_thresh
    
    """
    if width == 0 or height == 0:
        # Invalid size, treat as strip 
        return True

    ratio1 = width / height
    ratio2 = height / width
    return (ratio1 > ratio_thresh) or (ratio2 > ratio_thresh)


def is_all_nodata_or_constant(src):
    """
    Check if the raster is entirely NoData or constant value.

    Details:
    - Only band 1 is checked (LST is usually band 1).
    - Use masked=True so NoData pixels are automatically masked out.

    """
    # Read band 1 as masked array (NoData -> masked)
    arr = src.read(1, masked=True)

    # Count of valid (unmasked) pixels
    valid_count = arr.count()

    if valid_count == 0:
        # All pixels are NoData
        return True

    # Extract valid pixels as 1D array
    data = arr.compressed()
    if data.size == 0:
        # Safety check 
        return True

    # If min == max -> constant raster
    if np.min(data) == np.max(data):
        return True

    return False


def clean_rasters(input_dir, output_dir):
    """
    Main cleaning workflow.

    Steps:
        1. Loop over all .tif in input_dir.
           
        2. Skip long strips and all-NoData/constant rasters.
           
        3. Copy remaining rasters into output_dir.
           
    """
    os.makedirs(output_dir, exist_ok=True)

    # List all tif files in input_dir
    tif_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".tif")]
    tif_files.sort()

    kept = 0                    # number of kept rasters 
    removed_strip = 0           # removed long strips 
    removed_empty = 0           # removed empty/constant 

    print(f"Found {len(tif_files)} tif files in {input_dir}.")

    for i, fname in enumerate(tif_files, start=1):
        in_path = os.path.join(input_dir, fname)
        print(f"[{i}/{len(tif_files)}] Checking {fname} ...")

        try:
            with rasterio.open(in_path) as src:
                width = src.width
                height = src.height

                # 1) Check if the raster is a long strip
                if is_long_strip(width, height):
                    print(f"  -> Skip (long strip: {width} x {height})")
                    removed_strip += 1
                    continue

                # 2) Check if the raster is all NoData or constant
                if is_all_nodata_or_constant(src):
                    print("  -> Skip (all NoData or constant raster)")
                    removed_empty += 1
                    continue

            # 3) If passed both checks, copy to output_dir
            out_path = os.path.join(output_dir, fname)
            shutil.copy2(in_path, out_path)
            kept += 1
            print(f"  -> Kept and copied to {out_path}")

        except Exception as e:
            # If reading fails, just print error and move on
            print(f"  !! Error reading {fname}: {e}")

    # Summary 
    print("\nCleaning finished.")
    print(f"  Kept files             : {kept}")
    print(f"  Removed long strips    : {removed_strip}")
    print(f"  Removed empty/constant : {removed_empty}")
    print(f"\nClean rasters saved in:{output_dir}")


if __name__ == "__main__":
    clean_rasters(INPUT_DIR, OUTPUT_DIR)


Found 337 tif files in data\ecostress_philly_LST_clip.
[1/337] Checking 2018-07-29_01-09-04_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2018-07-29_01-09-04_LST_clip.tif
[2/337] Checking 2018-07-30_00-17-14_LST_clip.tif ...
  -> Skip (long strip: 55 x 429)
[3/337] Checking 2018-08-04_22-18-32_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[4/337] Checking 2018-08-04_22-18-32_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2018-08-04_22-18-32_LST_clip.tif
[5/337] Checking 2018-08-04_22-19-24_LST_clip.tif ...
  -> Skip (long strip: 55 x 429)
[6/337] Checking 2018-08-07_21-19-02_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[7/337] Checking 2018-08-07_21-19-02_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[8/337] Checking 2018-08-08_20-27-26_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[9/337] Checking 2018-08-08_20-27-26_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[10/337] Checking 2018-

  -> Kept and copied to data\ecostress_philly_LST_clean\2020-07-14_22-04-36_LST_clip.tif
[82/337] Checking 2020-07-29_23-02-39_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[83/337] Checking 2020-08-02_21-29-16_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[84/337] Checking 2020-08-02_21-29-16_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2020-08-02_21-29-16_LST_clip.tif
[85/337] Checking 2020-08-07_19-07-40_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[86/337] Checking 2020-09-08_00-09-16_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[87/337] Checking 2020-09-08_00-09-16_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2020-09-08_00-09-16_LST_clip.tif
[88/337] Checking 2020-09-11_22-35-53_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[89/337] Checking 2020-09-11_22-35-53_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[90/337] Checking 2020-09-14_21-50-13_LST_01_clip.tif ...
 

  -> Skip (long strip: 55 x 429)
[191/337] Checking 2022-09-19_19-12-40_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2022-09-19_19-12-40_LST_clip.tif
[192/337] Checking 2022-09-21_00-53-01_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[193/337] Checking 2022-09-21_00-53-53_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[194/337] Checking 2022-09-21_00-53-53_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2022-09-21_00-53-53_LST_clip.tif
[195/337] Checking 2023-07-03_01-48-31_LST_01_clip.tif ...
  -> Skip (all NoData or constant raster)
[196/337] Checking 2023-07-03_01-48-31_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[197/337] Checking 2023-07-10_22-36-54_LST_01_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2023-07-10_22-36-54_LST_01_clip.tif
[198/337] Checking 2023-07-10_22-36-54_LST_02_clip.tif ...
  -> Skip (long strip: 55 x 429)
[199/337] Checking 2023-07-10_22-36-54_LST_03

  -> Kept and copied to data\ecostress_philly_LST_clean\2025-07-11_20-36-21_LST_clip.tif
[297/337] Checking 2025-07-11_20-37-13_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[298/337] Checking 2025-07-11_20-37-13_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[299/337] Checking 2025-07-12_03-06-27_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[300/337] Checking 2025-07-12_03-06-27_LST_clip.tif ...
  -> Kept and copied to data\ecostress_philly_LST_clean\2025-07-12_03-06-27_LST_clip.tif
[301/337] Checking 2025-07-13_02-17-44_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[302/337] Checking 2025-07-13_02-17-44_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[303/337] Checking 2025-07-19_00-39-15_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[304/337] Checking 2025-07-19_23-50-37_LST_01_clip.tif ...
  -> Skip (long strip: 55 x 429)
[305/337] Checking 2025-07-19_23-50-37_LST_clip.tif ...
  -> Skip (all NoData or constant raster)
[306/3

## 0.5 Filter LST rasters by valid pixel ratio

In [33]:
"""
Filter LST rasters by valid pixel ratio.

"""

import os
import shutil

import numpy as np
import rasterio


# ========== Path settings ==========

# Input: cleaned LST rasters
INPUT_DIR = "data\ecostress_philly_LST_clean"

# Output: rasters with enough valid pixels
OUTPUT_DIR = "data\ecostress_philly_LST_valid30"


# Threshold for valid pixel ratio (0~1), e.g. 0.5 = 50%
MIN_VALID_FRAC = 0.3


# =================================================


def compute_valid_ratio(src):
    """
    Compute the ratio of valid pixels in band 1.
    
    """

    # Read band 1 as masked array; NoData pixels are masked
    arr = src.read(1, masked=True)

    total_pixels = arr.size          
    valid_pixels = arr.count()      

    if total_pixels == 0:
        return 0.0

    return valid_pixels / float(total_pixels)


def filter_by_valid_ratio(input_dir, output_dir, min_valid_frac=MIN_VALID_FRAC):
    """
    Main filtering workflow.
    """
    os.makedirs(output_dir, exist_ok=True)

    # 列出所有 .tif 文件 / list all .tif files
    tif_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".tif")]
    tif_files.sort()

    kept = 0
    removed = 0

    print(f"Found {len(tif_files)} tif files in {input_dir}.")
    print(f"Minimum valid pixel ratio: {min_valid_frac:.2f}")

    for i, fname in enumerate(tif_files, start=1):
        in_path = os.path.join(input_dir, fname)
        print(f"[{i}/{len(tif_files)}] Checking {fname} ...")

        try:
            # open raster and compute ratio
            with rasterio.open(in_path) as src:
                valid_ratio = compute_valid_ratio(src)

            print(f"  Valid ratio: {valid_ratio:.3f}")

            if valid_ratio >= min_valid_frac:
                # keep and copy
                out_path = os.path.join(output_dir, fname)
                shutil.copy2(in_path, out_path)
                kept += 1
                print("  -> Kept (ratio >= threshold)")
            else:
                # discard
                removed += 1
                print("  -> Removed (ratio < threshold)")

        except Exception as e:
            # If reading fails, treat as removed and print message
            removed += 1
            print(f"  !! Error reading {fname}: {e} ")


    print("\nFiltering finished.")
    print(f"  Kept files   : {kept}")
    print(f"  Removed files: {removed}")
    print(f"\nFinal rasters saved in：\n  {output_dir}")


if __name__ == "__main__":
    filter_by_valid_ratio(INPUT_DIR, OUTPUT_DIR, MIN_VALID_FRAC)


Found 92 tif files in data\ecostress_philly_LST_clean.
Minimum valid pixel ratio: 0.30
[1/92] Checking 2018-07-29_01-09-04_LST_clip.tif ...
  Valid ratio: 0.004
  -> Removed (ratio < threshold)
[2/92] Checking 2018-08-04_22-18-32_LST_clip.tif ...
  Valid ratio: 0.442
  -> Kept (ratio >= threshold)
[3/92] Checking 2018-09-07_02-27-26_LST_clip.tif ...
  Valid ratio: 0.142
  -> Removed (ratio < threshold)
[4/92] Checking 2018-09-16_22-37-17_LST_clip.tif ...
  Valid ratio: 0.357
  -> Kept (ratio >= threshold)
[5/92] Checking 2018-09-27_01-16-42_LST_clip.tif ...
  Valid ratio: 0.001
  -> Removed (ratio < threshold)
[6/92] Checking 2019-07-04_03-34-04_LST_clip.tif ...
  Valid ratio: 0.423
  -> Kept (ratio >= threshold)
[7/92] Checking 2019-07-14_23-22-15_LST_clip.tif ...
  Valid ratio: 0.415
  -> Kept (ratio >= threshold)
[8/92] Checking 2019-07-15_22-32-26_LST_clip.tif ...
  Valid ratio: 0.197
  -> Removed (ratio < threshold)
[9/92] Checking 2019-08-02_22-17-47_LST_clip.tif ...
  Valid rati